In [2]:
import pandas as pd

In [9]:
sp500_file = pd.read_csv('input_data/sp500_2022.csv')

In [16]:
firm_data = pd.read_csv('input_data/Firm_data.csv')
data_desc = pd.read_csv('data_description/Firm_data_descriptions.csv')

In [43]:
data_desc['Variable Name'] = data_desc['Variable Name'].str.lower()
data_desc = data_desc[['Variable Name','Description']]
column_names = pd.DataFrame(firm_data.columns)
column_names = column_names.rename(columns = {column_names.columns[0]: 'Variable Name'})
data_definitions = column_names.merge(data_desc, 
                how='left',
                on='Variable Name',
                indicator=False,
                validate='1:1')

In [44]:
data_definitions

,Variable Name,Description
0,gvkey,GVKEY -- Global Company Key (GVKEY)
1,datadate,NaN
2,fyear,FYEAR -- Data Year - Fiscal (FYEAR)
3,indfmt,NaN
4,consol,NaN
5,popsrc,NaN
6,datafmt,NaN
7,tic,Ticker Symbol (TIC)
8,cusip,CUSIP (CUSIP)
9,conm,Company Name (CONM)


# NOTE: EDA IS NEEDED! There are a ton of missing values throughout this dataset

In [53]:
def mtb(tic, year):
    market_value = firm_data.query('tic == @tic & fyear == @year')['mkvalt']
    book_value = firm_data.query('tic == @tic & fyear == @year')['ceql']
    return market_value / book_value

For Tobin's Q -- is "debt" just liabilities? or just debt? (I included both in the dataset)

In [56]:
def tobinsQ(tic, year):
    market_value = firm_data.query('tic == @tic & fyear == @year')['mkvalt']
    liq_pref = firm_data.query('tic == @tic & fyear == @year')['pstkl']
    debt = firm_data.query('tic == @tic & fyear == @year')['lt']
    assets = firm_data.query('tic == @tic & fyear == @year')['at']
    return (market_value+liq_pref+debt)/assets

In [ ]:
def rdScaled(tic, year

In [57]:
tobinsQ('AAPL',2012)

202    3.887248
dtype: float64